In [2]:
# -*- coding: utf-8 -*-
"""
Alpaca API - Calculate and store Relative Strength Index using historical data (V2 API)

"""

import os
import json
from alpaca_trade_api.rest import REST, TimeFrame
import numpy as np

API_KEY = os.getenv("ALPACA_API_KEY")
API_SECRET = os.getenv("ALPACA_SECRET_KEY")

endpoint = "https://paper-api.alpaca.markets"

headers = {'APCA-API-KEY-ID': API_KEY,
           'APCA-API-SECRET-KEY': API_SECRET}

def hist_data(symbols, start_date ="2021-12-01", timeframe="Minute"):
    """
    returns historical bar data for a list of tickers e.g. symbols = ["MSFT,AMZN,GOOG"]
    """
    df_data = {}
    api = REST(headers["APCA-API-KEY-ID"], headers["APCA-API-SECRET-KEY"], base_url=endpoint)
    for ticker in symbols:
        if timeframe == "Minute":
            df_data[ticker] = api.get_bars(ticker, TimeFrame.Minute, start_date, adjustment='all').df
        elif timeframe == "Hour":
            df_data[ticker] = api.get_bars(ticker, TimeFrame.Hour, start_date, adjustment='all').df
        else:
            df_data[ticker] = api.get_bars(ticker, TimeFrame.Day, start_date, adjustment='all').df
    return df_data
        
data_dump = hist_data(["FB","CSCO","AMZN"], start_date ="2021-12-15", timeframe="Hour") 


def RSI(df_dict, n=14):
    "function to calculate RSI"
    for df in df_dict:
        df_dict[df]["change"] = df_dict[df]["close"] - df_dict[df]["close"].shift(1)
        df_dict[df]["gain"] = np.where(df_dict[df]["change"]>=0, df_dict[df]["change"], 0)
        df_dict[df]["loss"] = np.where(df_dict[df]["change"]<0, -1*df_dict[df]["change"], 0)
        df_dict[df]["avgGain"] = df_dict[df]["gain"].ewm(alpha=1/n, min_periods=n).mean()
        df_dict[df]["avgLoss"] = df_dict[df]["loss"].ewm(alpha=1/n, min_periods=n).mean()
        df_dict[df]["rs"] = df_dict[df]["avgGain"]/df_dict[df]["avgLoss"]
        df_dict[df]["rsi"] = 100 - (100/ (1 + df_dict[df]["rs"]))
        df_dict[df].drop(["change","gain","loss","avgGain","avgLoss","rs"], axis=1, inplace=True)

RSI(data_dump)
        
      

{'FB':                               open    high     low   close   volume  \
timestamp                                                             
2021-12-15 09:00:00+00:00  334.000  334.00  331.50  332.00     5819   
2021-12-15 10:00:00+00:00  332.000  332.50  332.00  332.00     5782   
2021-12-15 11:00:00+00:00  331.500  332.00  331.50  332.00     1413   
2021-12-15 12:00:00+00:00  332.420  333.50  332.30  332.85     5425   
2021-12-15 13:00:00+00:00  332.910  334.32  332.54  332.65    20332   
...                            ...     ...     ...     ...      ...   
2022-06-08 19:00:00+00:00  196.720  197.47  195.82  196.63  3580091   
2022-06-08 20:00:00+00:00  196.640  196.64  196.02  196.37  1603233   
2022-06-08 21:00:00+00:00  196.573  196.64  196.11  196.21    11569   
2022-06-08 22:00:00+00:00  196.320  196.46  196.15  196.42     4661   
2022-06-08 23:00:00+00:00  196.450  196.50  196.12  196.29     5997   

                           trade_count        vwap        rsi  
times